In [ ]:
from warnings import filterwarnings
from os import getcwd
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor

from dspf.factor_analysis import FactorAnalysis
from dspf.factor_analysis.plotting import scree_plot, create_loadings_heatmaps
from factor_analyzer.factor_analyzer import calculate_kmo

filterwarnings("ignore")
np.set_printoptions(suppress=True, precision=4)
pd.set_option("display.precision", 4)

# Case Study: California Housing Data (1990)

Each observation (row) in the dataset represents what is called a [*block group*](https://www.census.gov/programs-surveys/geography/about/glossary.html#par_textimage_4) in California.

A 'block group' is a statistical division from the U.S. Census Bureau,
which should include between 600 and 3000 people.
Instead of block group, we will mostly use the term *district* here for simplicity.

## The data set

In [ ]:
path = Path(getcwd(), "data", "cal_housing.data")
X = pd.read_csv(path, header=0, sep=",")
X.info()

In total, we have 20640 observations (districts) and 9 metric scaled features.

Since we also need metrically scaled features for the factor analysis, we have no problems in this area.

In [ ]:
X.describe()

The features can thus be described as follows:

- `longitude`: longitude of a district, values between -124.35 and -114.31
- latitude`: latitude of a district, values between -32.54 and 41.95
- `housing_median_age`: median age of houses in a district, values between 1 and 52 (years)
- `total_rooms`: total number of rooms in a district, values between 2 and 39320
- `total_bedrooms`: total number of bedrooms in a district, values between 1 and 6445
- population`: total number of inhabitants of a district, values between 3 and 35682
- `households`: total number of households in a district, values between 1 and 6082
- `median_income`: median income of the inhabitants, values between 0.5 and 15
- median_house_value`: median of the monetary value of the houses, values between 15000 and 500 001 dollars.

The range of values of `median_income` is particularly striking. This is probably an adjusted scale and not income in dollars.

If we look again at the description of a district, we see that in this data set there are
there might be some outliers in terms of `population`. Because a district here should be between 600 and 3000 people.

### Missing values

The dataset does not contain any missing values, as we can see in the `pd.info` method.

## The distribution of the data

Now let's look at some univariate and bivariate plots of the metric features to get a feel for the data.

In [ ]:
X.hist(figsize=(20, 15), bins=30)
plt.show()


Here we see that the four features `total_rooms`, `total_bedrooms`, `population` and `households` have a relatively high
skewness.
Similarly, `median_house_value` has a high density at circa 500 000. This could be due to the fact
that the value 500 000 was used as an upper bound.


### Outlier Analysis
So before we look at bivariate plots, we will use LocalOutlierFactor (LOF) to try to eliminate some outliers.

In [ ]:
labels = LocalOutlierFactor(n_neighbors=35).fit_predict(X)
outliers = X[labels == -1]
X = X[labels == 1]

print(f"Number of districts identified as outliers by means of LOF: {outliers.shape[0]}")

In [ ]:
X.describe()

We can see that the maximum value of 'population' is now significantly lower. Nevertheless, there are districts with very small populations (minimum 3 inhabitants).

Let us now take a closer look at the distributions of the features by means of a pairplot.

In [ ]:
sns.pairplot(X, diag_kind="kde", kind="scatter", plot_kws={"alpha": 0.3})
plt.show()

In this pairplot, univariate plots are on the main diagonal and bivariate plots are on the off-diagonal elements.
Three groups in particular stand out here:

##### Group 1
The features `longitude` and `latitude` show a relatively high negative correlation.


##### Group 2
The features `total_rooms`, `total_bedrooms`, `population` and `households` are highly correlated with each other. However, this is
not surprising considering the meaning of the features, since a high number of people in the district must eventually mean that more
rooms exist and overall the number of households must probably be higher.

The feature `housing_median_age` could also still be included in this group, as it shows a slightly negative correlation with these four features.

##### Group 3
The features `median_income` and `median_house_value` also show a visible positive correlation, but not as strong as those in group 2.

These three groups are shown again below in separate pairplots.

In [ ]:
group1 = ["longitude", "latitude"]
group2 = ["total_rooms", "total_bedrooms", "population", "households"]
group3 = ["median_income", "median_house_value"]

In [ ]:
sns.pairplot(X[group1], diag_kind="kde", kind="scatter", plot_kws={"alpha": 0.1}, size=4)
plt.show()

Here we see that very many districts are located in two regions. This explains the bimodal structure of the two features 'longitude' and 'latitude'.
Below is a scatter plot that shows the geographic distribution of the districts in more detail.

In [ ]:
sns.pairplot(X[group2], diag_kind="kde", kind="scatter", plot_kws={"alpha": 0.3}, size=3)
plt.show()

In [ ]:
sns.pairplot(X[group3], diag_kind="kde", kind="scatter", plot_kws={"alpha": 0.3}, size=4)
plt.show()

In the last pairplot, the horizontal (or vertical) line at `median_house_value = 500 000` stands out again.


#### Geographical distribution of house prices

In addition, we can still look at the graphical distribution of house prices.
Shown below is a scatter plot, which takes into account the location of the districts in dependence
of the median house prices.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
sc = ax.scatter(x="longitude", y="latitude", c="median_house_value", cmap="Blues", data=X, alpha=0.4)
plt.colorbar(sc, label="median_house_value")

cities = ["Los Angeles", "San Francisco"]
xys = [(-118.243683, 34.052235), (-122.431297, 37.773972)]
xys_text = [(-121, 33.5), (-123, 35.5)]
for city, xy, xytext in zip(cities, xys, xys_text):
    ax.annotate(city, xy=xy,  xycoords='data',
            xytext=xytext,
            arrowprops=dict(facecolor='red', shrink=0.05),
            horizontalalignment='right', verticalalignment='top',
            )
plt.tight_layout()
plt.grid()
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
plt.show()

The districts with very expensive houses are at the bottom, that is, on the coast. Two large cities are marked with an arrow in the picture.


# Factor Analysis Step 1: Examine the Suitability of the Data

First, we need to examine the suitability of the data for factor analysis.
For this purpose, we use the Kaiser-Meyer-Olkin criterion (KMO criterion) for the complete data set and the related
related Measure of sampling adequacy (MSA) for each individual feature.

In general, we want the KMO value to be above 0.5 and the MSA value for each variable not to be below 0.5.
below 0.5.

In [ ]:
msa_values, kmo = calculate_kmo(X)
print(f"The KMO value is {kmo:.4f}\n")
msa_df = pd.DataFrame(msa_values.reshape(-1, 1), index=X.columns, columns=["MSA"])
print(msa_df)

The KMO value is above 0.6, which indicates acceptable quality. However, this is not an ideal value.
Also, the MSA value for 4 variables is below 0.5, but the value for the other features is good.

Now, for example, one could remove the features with an MSA value below 0.5. For this example
we will continue with all variables.
We should also look directly at the correlation matrix. We do this in the following with a
heatmap, since it is very well suited as a visual representation.


In [ ]:
fig = plt.figure(figsize=(10, 10))
corr = X.corr().round(2)
hm = sns.heatmap(data=corr, vmax=1, vmin=-1, cmap="RdBu_r", annot=True)
hm.set_xticklabels(X.columns, rotation=45)
hm.set_yticklabels(X.columns)
plt.tight_layout()
plt.show()

As we have already seen in the pairplot, there are three groups of highly correlated features:
The four features in the middle, and `housing_median_age` with a slightly negative correlation to these four features.
Similarly, `longitude` and `latitude` are negatively correlated with each other. The features `median_income` and `median_house_value` are positively correlated with each other.

This already suggests that a 3-factor solution might probably be a good choice of k. In the following, we will look at this in more detail.

# Factor Analysis Step 2: Choosing the Number of Factors.

To answer this question, we will first extract all factors using the Principal Component (PC) method and consider the eigenvalues of the factors using a scree plot.
and look at the eigenvalues of the factors using a scree plot.

Then we can keep the factors that have an eigenvalue greater than one (Kaiser criterion), or use a 'kink' in the plot to identify a suitable factor number.

In [ ]:
fa = FactorAnalysis(n_factors=X.shape[1], method="pc").fit(X)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
scree_plot(fa.eigenvalues_, ax)
ax.set_xlabel("Factor")
ax.set_ylabel("Eigenvalue")
plt.tight_layout()
plt.grid()
plt.show()

Here we can see that the first three factors have an eigenvalue
greater than one. This makes sense with respect to the correlation matrix,
which we saw earlier, because of the three 'boxes'. According to
the Kaiser criterion, we should therefore use a 3-factor model.

The fourth factor, however, is only minimally below the eigenvalue of one, which is why
one should not exclude it directly. A 'kink' would be visible at
`k = 5` recognizable.

# Factor analysis Step 3: Extracting the factors
Now we will perform the actual factor extraction.

For this we will compare three different extraction methods:
 - Principal Components (PC) Method.
 - Principal Axis Factoring (PAF) Method
 - Iterated Principal Axis Factoring (Iterated PAF)

The last variant is probably the most frequently used method (among these three).


In [ ]:
methods = [
    ("PC", FactorAnalysis(method="pc")),
    ("Non-iterated PAF", FactorAnalysis(method="paf", max_iter=1)),
    ("Iterated PAF", FactorAnalysis(method="paf", max_iter=50))
]
for n_factors in range(3, 6):
    figsize = (10 + (1+n_factors)//2, 8)
    create_loadings_heatmaps(X, methods, figsize, fa_params={"n_factors": n_factors})
    plt.gcf().suptitle(f"Loadings matrices of a {n_factors}-factor-model")
    plt.show()

Here we see in each row the charge matrices of the three different methods shown as a heat map.
We note that the 3-factor solution is indeed a good solution in terms of a simple structure.
However, the 4-factor solution could still be a valid solution as well. Only the 5-factor solution is problematic,
since no feature loads high (absolute value greater than 0.5) on the fifth factor.

One problem, however, is that the feature `housing_median_age` loads very high on the fourth factor only with the PC method
and has only a moderate loading on the first factor for a 3-factor solution.
This indicates a high specific variance, i.e., the factors are not well able to explain the variance of this feature
explain the variance of this feature.

So we will look at the 3-factor solution in more detail.

In [ ]:
fa_params = {"n_factors": 3}

axes = create_loadings_heatmaps(X, methods, figsize=(10, 9), fa_params=fa_params, annotate=True)
plt.tight_layout()
plt.show()

Wir können sehen, dass bei der PC-Methode die Ladungen leicht höher ausfallen und dass der zweite Faktor ein unterschiedliches
Vorzeichen besitzt, im Gegensatz zu den anderen beiden Methoden.

The iterated and non-iterated PAF methods are very similar to each other. However, the iterated
variant is often better in terms of the reproduced correlation matrix. This can be seen from the
*root mean squared error* (RMSE):

In [ ]:
for method, fa in methods:
    rmse = fa.get_rmse()
    print(f"RMSE of {method}: {rmse:.4f}")

Der root mean squared error of residuals (RMSE) ist bei der iterierten PAF-Methode am geringsten,
gefolgt von der nicht-iterativen Variante und der Hauptkomponentenmethode.

Schauen wir uns nun noch die Kommunalitäten der Variablen an. Dies zeigt uns zum Beispiel die `print_summary` Methode an:

In [ ]:
# Zusammenfassung der iterierten PAF-Methode
methods[2][1].print_summary()

The specific variance of `housing_median_age` is very high with a value of 0.8980.
This means that the factors together cannot explain the variance of this characteristic very well.
This is also reflected in the low loadings on the three factors.
However, the specific variances on the remaining features are very low,
which is a good sign for the quality of the factor solution.


Since the feature `housing_median_age` has a very high specific variance (low commonality)
we can also look at a 3-factor model without this feature.
This can reduce the RMSE by almost 44% (in comparison are the iterated PAF methods),
so we remove this feature for further analysis.

In [ ]:
X_without_age = X.drop(columns="housing_median_age", axis=1)
fa_without_age = FactorAnalysis(n_factors=3).fit(X_without_age)
perc = 1 - fa_without_age.get_rmse() / rmse
print(f"The RMSE could be reduced by {perc:.2%} by removing the feature")


We can also analyze the difference between the methods in the RMSE graphically:

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(12, 6))
fitted_methods = [
    ("PC", FactorAnalysis(method="pc", n_factors=3).fit(X)),
    ("Iterated PAF", FactorAnalysis(method="paf", n_factors=3, max_iter=50).fit(X))
]
for ax, (method, fa) in zip(axes, fitted_methods):
    R = fa.corr_
    R_hat = fa.get_reprod_corr()
    abs_residuals = np.abs(R - R_hat)
    mask = np.triu(np.ones_like(R))
    ax.set_title(f"{method} (RMSE = {fa.get_rmse():.4f})", fontsize=11)
    s = sns.heatmap(abs_residuals.round(2), cmap="BuGn", ax=ax, cbar=False, annot=True, square=True, mask=mask)
    s.set_xticklabels(range(1, 10))
    s.set_yticklabels(range(1, 10), rotation=0)
fig.suptitle("Residual matrices of two extraction methods with k=3")
fig.tight_layout()
plt.show()

Before we move on to factor rotation and interpretation, we will compare the different initial estimates of the communalities in the (iterated) PAF method.
It might be interesting to see whether the choice of the initial estimate has an influence on the final loadings.

In [ ]:
paf_comparison_methods = [
    ("Non-iterated PAF", FactorAnalysis(method="paf", max_iter=1)),
    ("PAF with max_iter=3", FactorAnalysis(method="paf", max_iter=3)),
    ("Iterated PAF", FactorAnalysis(method="paf", max_iter=50)),
]
figsize = (8, 6)
initial_communality_estimates = {
    "smc": "Squared multiple correlations (SMC)",
    "mac": "Maximum absolute correlation (MAC)",
    "ones": "Ones"
}
for init_comm in initial_communality_estimates:
    print(f"Initial Estimate: {initial_communality_estimates[init_comm]}")
    create_loadings_heatmaps(X, paf_comparison_methods, figsize, fa_params={"n_factors": 3, "initial_comm" : init_comm})
    plt.show()
    print(f"Iterated PAF finished in {paf_comparison_methods[2][1].n_iter_} Iterations.")
    for method, fa in paf_comparison_methods:
        print(f"RMSE of {method}: {fa.get_rmse():.4f}")
    print("\n")

We can see that only small differences between the different initial estimates are
in the charges are detectable. Only in the non-iterated variant of the PAF method can we detect some differences, especially in the second factor.
differences, especially in the second factor. For example, the second factor here has a different
sign, but only when ones were used as the communality estimate. In this
case, the result is identical to the principal component method.

We note, however, that the iterated variant requires a different number of iterations until
the convergence criterion is reached. On this data set, it is slowest in the case of ones (10 iterations) and fastest
in the case of maximum absolute correlations (MAC) (only 3 iterations). However, if the convergence criterion is met, the loadings are
are largely identical for all three initial estimates.

# Factor Analysis Step 4: Factor Rotation and Interpretation
Now we rotate the loadings with the Varimax method and try to interpret the factors.

In [ ]:
methods = [
    ("Unrotated", FactorAnalysis(method="paf", rotation=None)),
    ("Varimax-Rotation", FactorAnalysis(method="paf", rotation="varimax"))
]
fa_params = {"n_factors": 3}
fig = create_loadings_heatmaps(X_without_age, methods, fa_params=fa_params)
plt.tight_layout()

If we had to interpret the factors, we could say that
- the first factor reflects the size of the district,
- the second factor takes into account the location of the district, and
- the third factor denotes the prosperity in the district.


# Factor analysis Step 5: Determine the factor values

As a final step, we can take a look at the estimated factor scores
. As the district `i`, how would it have rated the three factors described above?

The method used here to estimate the factor scores is the *regression method*,
which uses multivariate linear regression to estimate the factor scores.

In [ ]:
scores = FactorAnalysis(n_factors=3).fit_transform(X_without_age)
scores = pd.DataFrame(scores, columns=["Size", "Location", "Prosperity"])
scores.head()

In [ ]:
scores.std(axis=0)

Here, the factor scores do not have a unit variance or standard deviation of one because the PAF method with
squared multiple squares was used as the initial estimate.

However, if the principal component method is used, the factor scores have a standard deviation of one.

In [ ]:
scores = FactorAnalysis(method="pc", n_factors=3).fit_transform(X_without_age)
scores.std(axis=0)